```
Copyright 2024 Nikolai Körber. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# PerCo (Stable Diffusion) v0.27.0

## Install

In [ ]:
#!pip install bitsandbytes
#!pip install accelerate
#!pip install transformers==4.37.2
#!pip install vector-quantize-pytorch
#!pip install torchac
#!pip install Ninja
#!pip install onnxruntime-training
#!pip install tensorflow-datasets
#!pip install datasets
#!pip install torchvision
#!pip install compressai
#!pip install torchmetrics
#!pip install clean-fid
#!pip install lpips

In [ ]:
!accelerate config default

## TensorBoard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir /tf/notebooks/PerCo/res/cmvl_2024/logs/ --host 0.0.0.0 --port=6010

## Train

In [ ]:
# BS = train_batch_size * num_processes * gradient_accumulation_steps = 160
# --resume_from_checkpoint="latest" \
# --use_ema \
# --snr_gamma=5.0 \
# --gradient_checkpointing \
# --use_lpips \
# --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \

# note that prediction_type must equal config.py prediction_type
# caution: parsing arguments other than provided might be ignored
!accelerate launch --multi_gpu --num_processes=8 /tf/notebooks/PerCo/src/train_sd_perco.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1" \
  --validation_image "/tf/notebooks/PerCo/res/eval/kodim13.png" "/tf/notebooks/PerCo/res/eval/kodim23.png" \
  --allow_tf32 \
  --dataloader_num_workers=12 \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=20 \
  --gradient_accumulation_steps=1 \
  --num_train_epochs=5 \
  --max_train_steps 50000 \
  --validation_steps 500 \
  --prediction_type="v_prediction" \
  --checkpointing_steps 500 \
  --learning_rate=1e-05 \
  --adam_weight_decay=1e-2 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=10000 \
  --checkpoints_total_limit=2 \
  --output_dir="/tf/notebooks/PerCo/res/cmvl_2024"

## Compression

In [ ]:
# evaluate single image
!python /tf/notebooks/PerCo/src/compression_utils.py -V --model_path "/tf/notebooks/PerCo/res/cmvl_2024" compress /tf/notebooks/PerCo/res/eval/kodim13.png /tf/notebooks/PerCo/res/eval/kodim13.pkl

In [ ]:
# compress only
!python /tf/notebooks/PerCo/src/compression_utils.py --model_path "/tf/notebooks/PerCo/res/cmvl_2024" compress /tf/notebooks/PerCo/res/eval/kodim13.png /tf/notebooks/PerCo/res/eval/kodim13.pkl

In [ ]:
# decompress only
!python /tf/notebooks/PerCo/src/compression_utils.py --model_path "/tf/notebooks/PerCo/res/cmvl_2024" decompress /tf/notebooks/PerCo/res/eval/kodim13.pkl /tf/notebooks/PerCo/res/eval/kodim13_hat.png

## Eval Kodak

In [ ]:
# generate reconstructions, compute BPP, PSNR, MS-SSIM, LPIPS
!python /tf/notebooks/PerCo/src/compression_utils.py --model_path "/tf/notebooks/PerCo/res/cmvl_2024" evaluate_ds --mode 0 --in_dir "/tf/notebooks/PerCo/res/data/Kodak/" --out_dir "/tf/notebooks/PerCo/res/data/Kodak_gen/"

In [ ]:
# compute metrics (FID + KID)
!python /tf/notebooks/PerCo/src/compression_utils.py --model_path "/tf/notebooks/PerCo/res/cmvl_2024" evaluate_ds --mode 1 --out_dir "/tf/notebooks/PerCo/res/data/Kodak_gen/"

## Eval MS-COCO 30k

In [ ]:
# generate reconstructions, compute BPP, PSNR, MS-SSIM, LPIPS
!python /tf/notebooks/PerCo/src/compression_utils.py --model_path "/tf/notebooks/PerCo/res/cmvl_2024" evaluate_ds --mode 0 --in_dir "/tf/notebooks/PerCo/res/data/MSCOCO30k/" --out_dir "/tf/notebooks/PerCo/res/data/MSCOCO30k_gen/"

In [ ]:
# compute metrics (FID + KID)
!python /tf/notebooks/PerCo/src/compression_utils.py --model_path "/tf/notebooks/PerCo/res/cmvl_2024" evaluate_ds --mode 1 --out_dir "/tf/notebooks/PerCo/res/data/MSCOCO30k_gen/"